In [1]:
import os, sys
sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))
from pathlib import Path

import matplotlib.pyplot as plt
from pydicom import dcmread
import pandas as pd
import numpy as np
import json

from src.utils.utils import *

# Dicionário de metadados

## DDSM

In [ ]:
metadata_files = ['calc_case_description_test_set',
             'mass_case_description_train_set',
             'calc_case_description_train_set',
             'mass_case_description_test_set']

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_files: # Itera sobre os json de metados
    metadata_ddsm = load_json(current_meta,  "../../metadata/DDSM/metadata_csv_and_dicom")
    
    for current_meta_study in metadata_ddsm: # itera sobre os estudos
        tmp_meta = current_meta_study['metadata_dicom_files']
        
        all_tags = set()
        empty_tags = set()
        
        meta_dicom_files = tmp_meta['original'] + tmp_meta['cropped']
        
        for meta_dicom_file in meta_dicom_files: # Itera sobre os arquivos dicom no estudo
            for key, value in meta_dicom_file.items(): # Iterar sobre as tags no arquivo dicom
                all_tags = all_tags | set(meta_dicom_file.keys()) # união
                empty_tags = empty_tags | set([key for key, value in meta_dicom_file.items() 
                                               if meta_dicom_file[key] == ""]) # união
        update_count_tag(all_tags, dictionary_metadata)
        update_count_tag(empty_tags, dictionary_empty_meta) 
        
dictionary = {"all": dictionary_metadata, "empty": dictionary_empty_meta}
save_json("dictionary_metadata", dictionary, f"../../metadata/DDSM")

## CMMD

In [ ]:
metadata_cmmd = load_json("metadata_csv_and_dicom", "../../metadata/CMMD/")

dictionary_metadata = {}
dictionary_empty_meta = {}

for current_meta in metadata_cmmd: # Iterar sobre os estudos
    meta_dicom_files = current_meta['metadata_dicom_files']
    
    all_tags = set()
    empty_tags = set()
    
    for current_meta_dicom in meta_dicom_files: # Iterar sobre os arquivos dicom no estudo
        for key, value in current_meta_dicom.items(): # Iterar sobre as tags no arquivo dicom
            all_tags = all_tags | set(current_meta_dicom.keys()) # união
            empty_tags = empty_tags | set([key for key, value in current_meta_dicom.items() 
                                           if current_meta_dicom[key] == ""]) # união
    
    update_count_tag(all_tags, dictionary_metadata)
    update_count_tag(empty_tags, dictionary_empty_meta) 
      
dictionary = {"all": dictionary_metadata, "empty": dictionary_empty_meta}
save_json("dictionary_metadata_cmmd", dictionary, f"../../metadata/CMMD") 

## Merge Dicionários de tags

In [4]:
def merge_dictionary(left_dict, right_dict):
    list_keys = list(left_dict.keys()) + list(right_dict.keys())
    dictionary_merged = {}

    for key in list_keys:
        if key in set(left_dict.keys()) & set(right_dict.keys()): # key contido (left^right)
            dictionary_merged[key] = left_dict[key] + right_dict[key]
        elif key in left_dict.keys(): # key contido em left
            dictionary_merged[key] = left_dict[key]
        else: # key contido em right
            dictionary_merged[key] = right_dict[key]
            
    return dictionary_merged

dictionary_cmmd = load_json("dictionary_metadata_cmmd", "../../metadata/CMMD/")
dictionary_ddsm = load_json("dictionary_metadata_ddsm", "../../metadata/DDSM/")

full_dictionary = merge_dictionary(dictionary_cmmd['all'], dictionary_ddsm['all'])
full_empty = merge_dictionary(dictionary_cmmd['empty'], dictionary_ddsm['empty'])

dictionary = {'all': full_dictionary, 'empty': full_empty}
save_json("dictionary_metadata_ddsm", dictionary, f"../../metadata")